<a href="https://colab.research.google.com/github/Juliana001/Desafios_bootcamp_luizalabs/blob/main/Desafio%202/desafioPOO_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from abc import ABC, abstractmethod
from datetime import datetime
from typing import List, Optional

class Transacao(ABC):
    """Classe abstrata para transações bancárias"""

    @property
    @abstractmethod
    def valor(self) -> float:
        pass

    @property
    @abstractmethod
    def conta(self) -> 'Conta':
        pass

    @abstractmethod
    def registrar(self, conta: 'Conta') -> bool:
        pass


class Saque(Transacao):
    def __init__(self, valor: float):
        self._valor = valor

    @property
    def valor(self) -> float:
        return self._valor

    @property
    def conta(self) -> Optional['Conta']:
        # Este método não faz sentido aqui, será sobrescrito no registro
        return None

    def registrar(self, conta: 'Conta') -> bool:
        sucesso_transacao = conta.sacar(self.valor)
        if sucesso_transacao:
            conta.historico.adicionar_transacao(self)
        return sucesso_transacao


class Deposito(Transacao):
    def __init__(self, valor: float):
        self._valor = valor

    @property
    def valor(self) -> float:
        return self._valor

    @property
    def conta(self) -> Optional['Conta']:
        return None

    def registrar(self, conta: 'Conta') -> bool:
        sucesso_transacao = conta.depositar(self.valor)
        if sucesso_transacao:
            conta.historico.adicionar_transacao(self)
        return sucesso_transacao


class Historico:
    def __init__(self):
        self._transacoes: List[dict] = []

    @property
    def transacoes(self) -> List[dict]:
        return self._transacoes.copy()  # Retorna cópia para proteger dados

    def adicionar_transacao(self, transacao: Transacao) -> None:
        transacao_data = {
            "tipo": transacao.__class__.__name__,
            "valor": transacao.valor,
            "data": datetime.now().strftime("%d-%m-%Y %H:%M:%S"),
        }
        self._transacoes.append(transacao_data)

    def gerar_relatorio(self, tipo_transacao: str = None) -> List[dict]:
        """Filtra transações por tipo se especificado"""
        if tipo_transacao:
            return [t for t in self._transacoes if t["tipo"] == tipo_transacao]
        return self._transacoes.copy()


class Cliente:
    def __init__(self, nome: str, endereco: str):
        self._nome = nome
        self._endereco = endereco
        self._contas: List['Conta'] = []

    @property
    def nome(self) -> str:
        return self._nome

    @property
    def endereco(self) -> str:
        return self._endereco

    @property
    def contas(self) -> List['Conta']:
        return self._contas.copy()

    def realizar_transacao(self, conta: 'Conta', transacao: Transacao) -> bool:
        """Realiza uma transação na conta especificada"""
        if conta not in self._contas:
            print(f"\n@@@ Operação falhou! Conta não pertence ao cliente. @@@")
            return False
        return transacao.registrar(conta)

    def adicionar_conta(self, conta: 'Conta') -> None:
        if conta not in self._contas:
            self._contas.append(conta)
            print(f"\n=== Conta {conta.numero} adicionada ao cliente {self.nome}. ===")

    def listar_contas(self) -> None:
        """Lista todas as contas do cliente"""
        if not self._contas:
            print("\n=== Nenhuma conta cadastrada. ===")
            return

        print(f"\n=== Contas de {self.nome} ===")
        for conta in self._contas:
            print(f"Agência: {conta.agencia} | Conta: {conta.numero} | Saldo: R$ {conta.saldo:.2f}")


class PessoaFisica(Cliente):
    def __init__(self, nome: str, endereco: str, cpf: str, data_nascimento: str):
        super().__init__(nome, endereco)
        self._cpf = cpf
        self._data_nascimento = data_nascimento

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> str:
        return self._data_nascimento

    def __str__(self) -> str:
        return f"{self.nome} (CPF: {self.cpf})"


class Conta(ABC):
    def __init__(self, numero: str, cliente: Cliente):
        self._saldo = 0.0
        self._numero = numero
        self._agencia = "0001"
        self._cliente = cliente
        self._historico = Historico()

    @classmethod
    def nova_conta(cls, cliente: Cliente, numero: str) -> 'Conta':
        return cls(numero, cliente)

    @property
    def saldo(self) -> float:
        return self._saldo

    @property
    def numero(self) -> str:
        return self._numero

    @property
    def agencia(self) -> str:
        return self._agencia

    @property
    def cliente(self) -> Cliente:
        return self._cliente

    @property
    def historico(self) -> Historico:
        return self._historico

    def sacar(self, valor: float) -> bool:
        """Realiza saque da conta"""
        if valor <= 0:
            print("\n@@@ Operação falhou! Valor inválido. @@@")
            return False

        if valor > self._saldo:
            print("\n@@@ Operação falhou! Saldo insuficiente. @@@")
            return False

        self._saldo -= valor
        print(f"\n=== Saque de R$ {valor:.2f} realizado com sucesso! ===")
        return True

    def depositar(self, valor: float) -> bool:
        """Realiza depósito na conta"""
        if valor <= 0:
            print("\n@@@ Operação falhou! Valor inválido. @@@")
            return False

        self._saldo += valor
        print(f"\n=== Depósito de R$ {valor:.2f} realizado com sucesso! ===")
        return True

    def extrato(self) -> None:
        """Exibe o extrato da conta"""
        print("\n" + "=" * 40)
        print(f"EXTRATO - Agência: {self.agencia} | Conta: {self.numero}")
        print(f"Cliente: {self.cliente.nome}")
        print("=" * 40)

        if not self._historico.transacoes:
            print("Não foram realizadas movimentações.")
        else:
            for transacao in self._historico.transacoes:
                print(f"{transacao['data']} - {transacao['tipo']}: R$ {transacao['valor']:.2f}")

        print(f"\nSaldo atual: R$ {self._saldo:.2f}")
        print("=" * 40)


class ContaCorrente(Conta):
    def __init__(self, numero: str, cliente: Cliente,
                 limite: float = 500.0, limite_saques: int = 3):
        super().__init__(numero, cliente)
        self._limite = limite
        self._limite_saques = limite_saques
        self._saques_realizados = 0

    @property
    def limite(self) -> float:
        return self._limite

    @property
    def limite_saques(self) -> int:
        return self._limite_saques

    @property
    def saques_realizados(self) -> int:
        return self._saques_realizados

    def sacar(self, valor: float) -> bool:
        """Realiza saque na conta corrente com limites"""
        if self._saques_realizados >= self._limite_saques:
            print("\n@@@ Operação falhou! Limite de saques diários atingido. @@@")
            return False

        if valor > self._limite:
            print(f"\n@@@ Operação falhou! Valor excede o limite de R$ {self._limite:.2f}. @@@")
            return False

        # Verifica saldo considerando limite
        if valor > (self._saldo + self._limite):
            print("\n@@@ Operação falhou! Saldo + limite insuficientes. @@@")
            return False

        sucesso = super().sacar(valor)
        if sucesso:
            self._saques_realizados += 1
        return sucesso

    def __str__(self) -> str:
        return f"""
        Agência:\t{self.agencia}
        Conta Corrente:\t{self.numero}
        Titular:\t{self.cliente.nome}
        Saldo:\t\tR$ {self.saldo:.2f}
        Limite:\t\tR$ {self.limite:.2f}
        Saques hoje:\t{self.saques_realizados}/{self.limite_saques}
        """


if __name__ == "__main__":
    # Criando cliente
    cliente = PessoaFisica(
        nome="João Silva",
        endereco="Rua A, 123",
        cpf="123.456.789-00",
        data_nascimento="01/01/1990"
    )

    # Criando conta corrente
    conta = ContaCorrente.nova_conta(cliente=cliente, numero="12345-6")
    cliente.adicionar_conta(conta)

    # Realizando operações
    deposito = Deposito(1000.0)
    cliente.realizar_transacao(conta, deposito)

    saque = Saque(200.0)
    cliente.realizar_transacao(conta, saque)

    # Exibindo extrato
    conta.extrato()

    # Listando contas do cliente
    cliente.listar_contas()

    # Exibindo informações da conta
    print(conta)


=== Conta 12345-6 adicionada ao cliente João Silva. ===

=== Depósito de R$ 1000.00 realizado com sucesso! ===

=== Saque de R$ 200.00 realizado com sucesso! ===

EXTRATO - Agência: 0001 | Conta: 12345-6
Cliente: João Silva
18-12-2025 20:48:21 - Deposito: R$ 1000.00
18-12-2025 20:48:21 - Saque: R$ 200.00

Saldo atual: R$ 800.00

=== Contas de João Silva ===
Agência: 0001 | Conta: 12345-6 | Saldo: R$ 800.00

        Agência:	0001
        Conta Corrente:	12345-6
        Titular:	João Silva
        Saldo:		R$ 800.00
        Limite:		R$ 500.00
        Saques hoje:	1/3
        
